# Heuristics for FFMSP

In [376]:
import numpy as np
import time

### Utilities

In [253]:
filename = "problem_instances/100-300-001.txt"
data = []
mapper = {'A':0, 'C':1, 'T':2, 'G':3} #char to int
rev_mapper = {0:'A', 1:'C' , 2:'T', 3:'G'} #int to char, whenever needed
alphabet = (0,1,2,3)
# read per char, for matrix data structure, while mapping ['A', 'C', 'T', 'G'] to [0,1,2,3] at the same time:
with open(filename) as fileobj:
    for line in fileobj:
        d = []
        line = line.rstrip("\n")
        for ch in line:
            mapch = mapper[ch]
            d.append(mapch)
        data.append(d)
n = len(data); m = len(data[0])
data = np.array(data)
#count = np.char.count(data[0], 'A')
count = np.count_nonzero(data == mapper['A'], axis=0)

print(data[:,0])
print(count)

[3 3 2 1 2 1 0 1 1 2 1 0 3 0 1 3 2 2 3 2 3 2 2 3 2 0 2 3 0 3 0 0 3 3 2 2 0
 0 3 0 0 1 1 3 2 2 2 3 3 3 0 2 2 0 0 1 1 1 0 2 2 2 3 1 0 1 0 0 3 1 2 3 3 1
 0 0 1 1 2 1 0 0 1 1 1 1 3 2 2 0 3 2 2 2 3 0 3 2 1 1]
[24 28 30 25 26 25 28 26 23 27 23 26 20 24 27 22 26 35 27 26 27 31 24 28
 27 18 21 29 24 30 20 25 21 26 30 28 24 24 30 24 28 32 22 24 27 29 29 25
 32 19 20 28 20 24 29 24 30 19 18 22 28 31 19 29 34 28 31 30 28 26 23 24
 19 28 31 22 21 33 25 30 25 17 23 32 26 26 22 25 31 26 21 33 26 25 23 23
 24 26 24 24 17 31 28 24 22 23 21 24 29 26 31 26 26 23 24 23 20 33 20 28
 23 22 26 26 34 25 31 29 29 31 27 31 28 24 20 27 31 25 24 29 32 19 31 26
 18 25 28 24 20 18 25 25 30 27 23 25 22 26 23 25 23 30 24 25 23 24 15 32
 22 16 24 19 25 26 25 20 23 28 25 26 22 21 26 26 25 26 20 34 25 33 26 30
 31 28 22 23 25 29 25 23 22 22 27 26 29 27 34 25 25 27 33 25 23 22 24 30
 23 27 27 28 26 25 21 30 18 25 31 24 28 18 36 25 21 16 20 30 24 24 28 27
 25 20 23 24 18 22 14 32 25 29 24 30 24 22 32 28 25 16 22 21 31 18

In [254]:
def ffmsp_obj(sol, data, threshold):
    '''objective function of the FFMSP:
    computes the hamming distance of the solution and each solution in the data,
    if the count of hamming distances of sol and one sol from data >= t, then add 1 cardinality.
    
    returns: objective function value, scalar.
    params:
        - sol, vector of solution of FFMSP, shape = m
        - data, matrix containing list of strings, shape = (n, m)
        - threshold, [0, m], scalar.
    '''
    # init vars:
    n = data.shape[0]; m = data.shape[1]
    y = 0
    
    # compute the hamming distance of one cell of sol to all in data:
    hamming_array = np.not_equal(sol, data) # contains matrix of predicate function
    #print(hamming_array)
    for i in range(n):
        count_hamming = 1 if np.sum(hamming_array[i]) >= threshold else 0
        y += count_hamming
    return y

In [255]:
sol = np.array([0,1,3,3])
dat = np.array([
                [0, 1, 3, 3],
                [0, 2, 0, 2],
                [3, 1, 3, 0],
                [0,2,3,3]
                ])
ffmsp_obj(sol, dat, 2)

2

### Greedy algo

In [256]:
def greedy(data, alphabet, t):
    '''
    the simple greedy algo for FFMSP uses a consensus of each column (string position): 
    takes the char with the least occurence from all strings on each position - maximization problem. 
    
    returns:
        - solution vector, shape = m
        - objective function, scalar
    params:
        - data, matrix containing list of strings, shape = (n, m)
        - alphabet, vector (mathematically, a set), shape = len(alphabet)
        - threshold t, scalar
    '''
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # since t is in percentage
    freq_mat = np.zeros((alpha_size, m))
    # count the occurences of each alphabet column wise:
    for i in range(alpha_size):
        freq_mat[i] = np.count_nonzero(data == alphabet[i], axis = 0) # alphabet[i] == i in this case, can use whichever
    
    #print(freq_mat)
    sol = np.argmin(freq_mat, axis=0) # get char with lowest frequency for each position [0, m]
    f = ffmsp_obj(sol, data, threshold) # compute obj fun
    return sol, f
    

In [257]:
print(dat)
greedy(dat, alphabet, 0.5)

[[0 1 3 3]
 [0 2 0 2]
 [3 1 3 0]
 [0 2 3 3]]


(array([1, 0, 1, 1]), 4)

In [344]:
greedy(data, alphabet, 0.85)

(array([0, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 1, 0, 3, 3, 0, 1, 2, 1, 1, 1, 1,
        2, 2, 2, 0, 3, 3, 2, 2, 0, 1, 2, 3, 2, 1, 1, 2, 1, 3, 3, 1, 3, 3,
        2, 3, 1, 3, 3, 0, 0, 3, 0, 3, 1, 1, 2, 0, 0, 3, 1, 3, 0, 2, 2, 3,
        3, 2, 1, 1, 1, 0, 0, 2, 3, 0, 0, 1, 1, 3, 3, 0, 3, 3, 1, 1, 1, 3,
        2, 2, 0, 1, 3, 1, 3, 2, 2, 2, 2, 3, 0, 1, 3, 0, 0, 0, 2, 2, 1, 2,
        2, 2, 3, 2, 3, 1, 0, 1, 0, 2, 1, 0, 1, 3, 3, 3, 2, 2, 2, 2, 2, 3,
        3, 0, 0, 3, 3, 1, 2, 3, 3, 0, 2, 3, 0, 2, 3, 2, 0, 0, 1, 3, 2, 3,
        3, 3, 1, 3, 1, 2, 2, 2, 0, 2, 1, 3, 0, 1, 0, 0, 3, 0, 2, 2, 1, 0,
        2, 3, 3, 2, 0, 0, 2, 1, 2, 1, 0, 1, 3, 3, 1, 2, 3, 2, 3, 3, 3, 2,
        3, 1, 0, 1, 2, 3, 1, 1, 2, 2, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 2,
        1, 3, 0, 2, 0, 3, 1, 2, 3, 0, 3, 3, 0, 0, 0, 2, 2, 1, 1, 2, 2, 0,
        1, 3, 0, 3, 0, 3, 3, 1, 2, 1, 1, 2, 1, 1, 2, 0, 3, 0, 1, 0, 1, 0,
        0, 2, 1, 0, 2, 1, 1, 0, 3, 1, 2, 0, 0, 2, 1, 1, 3, 0, 0, 0, 0, 3,
        0, 1, 0, 2, 3, 2, 0, 1, 3, 2, 

### Local search

In [416]:
def local_search(data, alphabet, t, init='greedy', sol=None):
    '''
    simple local search, by flipping each cell and only accepting solution(s) that are better
    pretty much gradient descent but FFMSP ver.
    
    returns the same params and takes in the same params as greedy algo except:
    params:
        - init, decides initialization mode, string
        - sol, starting solution, will use this instead of init if this is not empty, shape = m
    '''
    # init var:
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # for obj fun
    f=0
    
    # generate init sol:
    if sol is not None:
        f = ffmsp_obj(sol, data, threshold)
    else:
        if init == "greedy":
            sol, f = greedy(data, alphabet, t)
        elif init == "random":
            sol = np.random.randint(alpha_size, size=m)
            f = ffmsp_obj(sol, data, threshold)
    
    # do local search, flip each bit position:
    for i in range(m):
        for j in range(alpha_size):
            if sol[i] != alphabet[j]: # exclude current char
                # implicitly flip bit, check if better - if yes then stop and check next pos:
                sol_new = np.copy(sol) # need to copy since numpy by default refers to memory instead. Need to replace with more eficient op
                sol_new[i] = j
                f_new = ffmsp_obj(sol_new, data, threshold)
                #print(sol_new, f_new)
                #print(sol, f, i, j, sol_new, f_new)
                if f_new >= f:
                    sol = sol_new; f = f_new
                    #print("should break here!")
                    #break # without break seems better
    return sol, f

In [417]:
print(dat)
local_search(dat, alphabet, 0.5, init='greedy', sol = np.array([2,2,3,3]))

[[0 1 3 3]
 [0 2 0 2]
 [3 1 3 0]
 [0 2 3 3]]


(array([3, 3, 3, 3]), 4)

In [423]:
local_search(data, alphabet, 0.8, init='greedy')

(array([1, 2, 3, 3, 1, 2, 3, 3, 3, 1, 3, 3, 1, 3, 3, 2, 0, 3, 1, 3, 3, 1,
        2, 2, 2, 0, 3, 1, 2, 2, 0, 1, 2, 3, 2, 1, 1, 2, 1, 3, 3, 1, 3, 3,
        2, 3, 1, 3, 3, 0, 0, 3, 2, 3, 1, 1, 2, 0, 0, 3, 1, 3, 0, 2, 2, 3,
        3, 3, 1, 1, 3, 1, 0, 2, 3, 0, 0, 1, 1, 3, 3, 0, 3, 3, 1, 1, 1, 3,
        2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 1, 0, 1, 3, 0, 0, 0, 2, 2, 3, 2,
        2, 2, 3, 2, 3, 1, 0, 1, 0, 2, 1, 0, 1, 3, 3, 3, 2, 2, 2, 2, 2, 3,
        3, 0, 0, 3, 3, 1, 2, 3, 3, 0, 2, 3, 0, 2, 3, 2, 0, 0, 1, 3, 2, 3,
        3, 3, 1, 3, 1, 2, 2, 2, 0, 2, 1, 3, 0, 1, 0, 0, 3, 0, 2, 2, 1, 0,
        2, 3, 3, 2, 0, 0, 2, 1, 2, 1, 0, 1, 3, 3, 1, 2, 3, 2, 3, 3, 3, 2,
        3, 1, 0, 1, 2, 3, 1, 1, 2, 2, 3, 3, 3, 2, 0, 2, 2, 1, 3, 2, 2, 2,
        1, 3, 0, 2, 0, 3, 1, 2, 3, 0, 3, 3, 0, 0, 1, 2, 2, 1, 1, 2, 2, 0,
        1, 3, 0, 3, 0, 3, 3, 1, 2, 1, 1, 2, 1, 1, 2, 0, 3, 0, 1, 1, 1, 0,
        1, 2, 1, 0, 2, 1, 1, 0, 3, 1, 2, 0, 0, 2, 1, 1, 3, 0, 0, 0, 0, 3,
        0, 3, 0, 2, 3, 2, 0, 1, 3, 2, 

### Metaheuristic

In [424]:
def metaheuristic(data, alphabet, t, max_loop, rand, init="greedy"):
    '''
    simple idea: to avoid local minima, use perturbation - randomly swapping cells with random letters, where the random percentage is a tuning parameter  
    returns the same params as the greedy algo.
    params, same as prev methods, except:
        - max_loop, hyperparameter determining maximum perturbation+local search ops, int scalar
        - rand, a hyperparameter [0,1] percentage of the mutated cells, lower means faster convergence, real scalar 
            -> seems like near 0 is a good choice
            -> rand doesnt give any effect! maybe because of the local search is less strict
    '''
     # init var:
    n = data.shape[0]; m = data.shape[1]; alpha_size = len(alphabet)
    threshold = int(t*m) # for obj fun
    f=0
    
    # generate init sol:
    if init == "greedy":
        sol, f = greedy(data, alphabet, t)
    elif init == "random":
        sol = np.random.randint(alpha_size, size=m)
        f = ffmsp_obj(sol, data, threshold)
        
    # loop the local search and perturbation:
    i = 0
    perturb_length = int(rand*m)
    print(perturb_length)
    # do initial local search for the lower bound:
    sol, f = local_search(data, alphabet, t, sol=sol)
    while i<max_loop:
        # perturb sol, generate random integer array [0,alpha_size] with length = perturb_length which replaces random cells:
        part_sol = np.random.randint(alpha_size, size=perturb_length)
        idx = np.random.randint(m, size=perturb_length) # replacement indexes
        #print(part_sol, idx)
        sol_perturb = np.copy(sol) # i wonder if there's another way than just copying, seem heavy
        sol_perturb[idx] = part_sol # replace some sol components wiwth the part_sol
        #print(sol_perturb)
        # do local search:
        sol_new, f_new = local_search(data, alphabet, t, sol=sol_perturb)
        
        #sort of greedy acceptante criteria, compare with previous local minimum:
        if f_new >= f:
            sol = sol_new
            f = f_new
            print(i,"accepted",f)
        i+=1
    return sol, f

In [ ]:
start = time.time()
metaheuristic(data, alphabet, 0.8, 500, 1e-3, init="greedy")
elapsed = time.time()-start
print(elapsed)

0
0 accepted 67
1 accepted 68
2 accepted 68
3 accepted 68
4 accepted 68
5 accepted 68
6 accepted 68
7 accepted 68
8 accepted 68
9 accepted 68
10 accepted 68
11 accepted 68
12 accepted 68
13 accepted 68
14 accepted 68
15 accepted 68
16 accepted 68
17 accepted 68
18 accepted 68
19 accepted 68
20 accepted 68
21 accepted 68
22 accepted 68
23 accepted 68
24 accepted 68
25 accepted 68
26 accepted 68
27 accepted 68
28 accepted 68
29 accepted 68
30 accepted 68
31 accepted 68
32 accepted 68
33 accepted 68
34 accepted 68
35 accepted 68
36 accepted 68
37 accepted 68
38 accepted 68
39 accepted 68
40 accepted 68
41 accepted 68
42 accepted 68
43 accepted 68
44 accepted 68
45 accepted 68
46 accepted 68
47 accepted 68
48 accepted 68
49 accepted 68
50 accepted 68
51 accepted 68
52 accepted 68
53 accepted 68
54 accepted 68
55 accepted 68
56 accepted 68
57 accepted 68
58 accepted 68
59 accepted 68
60 accepted 68
61 accepted 68
62 accepted 68
63 accepted 68
64 accepted 68
65 accepted 68
66 accepted 68
67 

In [374]:
# compare with genetic algo:


1.2